In [1]:
!pip install torch
!pip install torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 MB 646.0 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.0 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 6.8 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 KB 3.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 KB 4.0 MB/s eta 0:00:0000:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.3.0
    Uninstalling typing_extensions-4.3.0:
      Successfully uninstalled typing_extensions-4.3.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 6.3 MB/s eta 0:00:00:00:0100:01


In [2]:
from torch import optim, nn
from collections import OrderedDict

hidden_units = [4, 12]
output_units = 10
model_d = nn.Sequential(OrderedDict([
    ('conv1_depthwise', nn.Conv2d(1, 1, 3, stride=3, padding=1, groups=1, bias=False)),
    ('conv1_pointwise', nn.Conv2d(1, hidden_units[0], 1, bias=False)),
    ('Relu1', nn.ReLU()),
    ('conv2_depthwise', nn.Conv2d(hidden_units[0], hidden_units[0], 3, stride=3, padding=1, groups=hidden_units[0], bias=False)),
    ('conv2_pointwise', nn.Conv2d(hidden_units[0], hidden_units[1], 1, bias=False)),
    ('Relu2', nn.ReLU()),
    ('conv3_depthwise', nn.Conv2d(hidden_units[1], hidden_units[1], 4, stride=4, padding=0, groups=hidden_units[1], bias=False)),
    ('conv3_pointwise', nn.Conv2d(hidden_units[1], output_units, 1, bias=False)),
    ('log_softmax', nn.LogSoftmax(dim = 1))
]))

In [3]:
import torch
model_d.load_state_dict(torch.load("assets/model.pth"))

<All keys matched successfully>

In [10]:
import json
images = json.load(open("assets/images_py.json"))
labels = json.load(open("assets/labels.json"))

In [13]:
images_t = []
for image in images:
    images_t.append(torch.tensor(image))

In [15]:
import time
total_correct = 0
total_num = 0
t0 = time.time()
for image, label in zip(images_t, labels):
  ps = torch.exp(model_d(image)).view(10)
  pred = torch.argmax(ps)
  total_correct += pred.item() == label
  total_num += 1
t1 = time.time()
print("Total elapsed time:", t1-t0, "s")
print("Elapsed time per picture:", ((t1-t0) / total_num) * 1000, "ms")
print('Accuracy:', total_correct / float(total_num))

Total elapsed time: 2.9771807193756104 s
Elapsed time per picture: 2.9074030462652445 ms
Accuracy: 0.9052734375
